In [1]:
import numpy as np
import pandas as pd
import datetime
import time
from tqdm import tqdm
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from scipy import stats
from math import sqrt
from sklearn.model_selection import GridSearchCV
%run ./../custom_functions/train_dev_test_split.ipynb
%run ./../custom_functions/get_future_predictions.ipynb

#from ipynb.fs.full.train_dev_test_split import train_dev_test_split
#from ipynb.fs.full.get_future_predictions import get_future_preds
#from ipynb.fs.full.get_scoring_measures import get_scoring_measures

from sklearn.ensemble import RandomForestRegressor

In [2]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from scipy import stats
from math import sqrt

In [3]:
marag_df = pd.read_csv('./../../../Databases/clean_data/marag_lags_data.csv')
marag_df['Timestamp'] = pd.to_datetime(marag_df['Timestamp'], dayfirst=True)
marag_df.set_index('Timestamp', inplace=True)
marag_df.sort_index(inplace=True)

marag_daily = pd.read_csv('./../../../Databases/clean_data/marag_dailylags_data.csv')
marag_daily.set_index('Timestamp', inplace=True)
marag_daily.sort_index(inplace=True)

# For the time being we will not use the outliers column, so we will drop it
marag_df.drop(['outliers'], axis=1, inplace=True)

In [4]:
# Let's import the marag data with no lags for future comparative purposes
no_lags_marag = pd.read_csv('./../../../Databases/clean_data/marag_data.csv')
no_lags_marag.set_index('Timestamp', inplace=True)
no_lags_marag.sort_index(inplace=True)
no_lags_marag.drop(['outliers'], axis=1, inplace=True)

daily_nolags_marag = pd.read_csv('./../../../Databases/clean_data/marag_daily_data.csv')
daily_nolags_marag.set_index('Timestamp', inplace=True)
daily_nolags_marag.sort_index(inplace=True)

In [5]:
# Let's create a dataframe in which we'll append all the estimated predictions, and another
# containing the running times of each scenario.

rf_results_df = pd.DataFrame
rf_runtime_df = pd.DataFrame

rf_dailyresults_df = pd.DataFrame
rf_runtime_dailydf = pd.DataFrame

## Model tuning

In [6]:
X_train, X_dev, X_test, y_train, y_dev, y_test = train_dev_test_split(marag_df.drop(['TotalEntries'], axis=1), 
                                                                      marag_df['TotalEntries'])
rftune_X_train = pd.concat([X_train, X_dev])
rftune_X_train.sort_index(inplace=True)
rftune_y_train = pd.concat([y_train, y_dev])
rftune_y_train.sort_index(inplace=True)

In [7]:
# RF baseline

In [8]:
rf = RandomForestRegressor(random_state = 42)
rf.fit(X_train, y_train)
RF_baseline_y_hat = rf.predict(X_dev)

/home/quark/anaconda3/envs/mpro/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [9]:
baseline_RMSE = pd.DataFrame(pd.Series(np.sqrt(mean_squared_error(y_dev, RF_baseline_y_hat)), name = 'RMSE'))
baseline_RMSE

,RMSE
0,33.85374


In [13]:
n_estimators = [100, 200, 400, 600]
max_depth = [10, 30, 60, 100]
best_RMSE = []

start_rf_tune = time.time()

for ne in tqdm(n_estimators):
    for md in tqdm(max_depth):
        rf = RandomForestRegressor(n_estimators = ne, max_depth = md, n_jobs = -1, random_state = 42)
        rf.fit(X_train, y_train)
        y_dev_hat = rf.predict(X_dev)
        RMSE_rfparams = sqrt(mean_squared_error(y_dev, y_dev_hat))
        
        if pd.DataFrame(best_RMSE).empty:
            best_RMSE = [ne, md, RMSE_rfparams]
        else:
            if RMSE_rfparams < best_RMSE[2]:
                best_RMSE = [ne, md, RMSE_rfparams]
        
end_rf_tune = time.time()
rf_tune_runtime = end_rf_tune - start_rf_tune



  0%|          | 0/4 [00:00<?, ?it/s]


  0%|          | 0/4 [00:00<?, ?it/s]


 25%|██▌       | 1/4 [01:25<04:17, 85.93s/it]


 50%|█████     | 2/4 [03:34<03:17, 98.70s/it]


 75%|███████▌  | 3/4 [05:45<01:48, 108.45s/it]


100%|██████████| 4/4 [07:58<00:00, 115.83s/it]

 25%|██▌       | 1/4 [07:58<23:56, 478.67s/it]


  0%|          | 0/4 [00:00<?, ?it/s]


 25%|██▌       | 1/4 [02:45<08:16, 165.62s/it]


 50%|█████     | 2/4 [07:01<06:25, 192.77s/it]


 75%|███████▌  | 3/4 [11:17<03:31, 211.67s/it]


100%|██████████| 4/4 [15:35<00:00, 225.67s/it]

 50%|█████     | 2/4 [23:34<20:31, 615.83s/it]


  0%|          | 0/4 [00:00<?, ?it/s]


 25%|██▌       | 1/4 [05:20<16:00, 320.09s/it]


 50%|█████     | 2/4 [13:52<12:35, 377.70s/it]


 75%|███████▌  | 3/4 [22:35<07:01, 421.41s/it]


100%|██████████| 4/4 [31:05<00:00, 447.97s/it]

 75%|███████▌  | 3/4 [54:40<16:30, 990.75s/it]


  0%|          | 0/4 [00:00<?, ?it/s]


 25%|██▌       | 1/4 [07:57<23:52, 477.61s/it]


 50%|█████     | 2/

In [14]:
print(best_RMSE, rf_tune_runtime)

[400, 30, 32.32575519464866] 6071.081792593002


In [63]:
# CAREFUL!!! Review below code


n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
hyperparameters_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

#cv = (np.array(X_train.index), np.array(X_dev.index))

For the hyperparameter tunning, it will be used GridSearchCV as a way to test a predefined grid of parameters, and the TimeSeriesSplit function as a generator for the cv parameter.

In [66]:
start_rf_tune = time.time()
cv = TimeSeriesSplit(n_splits=2).split(rftune_X_train)
rfr_tunned = GridSearchCV(estimator=rf, param_grid=hyperparameters_grid, cv=cv, verbose=3, n_jobs=-1, pre_dispatch='2*n_jobs')
rfr_tunned.fit(rftune_X_train, rftune_y_train)

end_rf_tune = time.time()
rf_tune_runtime = end_rf_tune - start_rf_tune

Fitting 2 folds for each of 4320 candidates, totalling 8640 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed: 203.8min
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed: 1151.5min


KeyboardInterrupt: 

In [ ]:
rfr_tunned.best_params_

In [15]:
# CAREFUL!! TO BE ERASED

rf = RandomForestRegressor(random_state = 42)
from pprint import pprint
# Look at parameters used by our current forest
print('Parameters currently in use:\n')
pprint(rf.get_params())

Parameters currently in use:

{'bootstrap': True,
 'criterion': 'mse',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 'warn',
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}


In [14]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)
{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}

NameError: name 'pprint' is not defined

In [18]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 2 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 2, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 2 folds for each of 100 candidates, totalling 200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed: 11.5min


KeyboardInterrupt: 

In [ ]:
rf_random.best_params_

In [ ]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy
base_model = RandomForestRegressor(n_estimators = 10, random_state = 42)
base_model.fit(X_train, y_train)
base_accuracy = evaluate(base_model, X_dev, y_dev)

In [ ]:
best_random = rf_random.best_estimator_
random_accuracy = evaluate(best_random, test_features, test_labels)

In [ ]:
print('Improvement of {:0.2f}%.'.format( 100 * (random_accuracy - base_accuracy) / base_accuracy))

## Alternative method creating custom idxs for cross validation:

https://stackoverflow.com/questions/37583263/scikit-learn-cross-validation-custom-splits-for-time-series-data

https://www.kaggle.com/sociopath00/random-forest-using-gridsearchcv

https://stackoverflow.com/questions/30102973/how-to-get-best-estimator-on-gridsearchcv-random-forest-classifier-scikit

In [43]:
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
my_cv = TimeSeriesSplit(n_splits=2).split(rftune_X_train)

In [47]:
my_cv.

<generator object TimeSeriesSplit.split at 0x7f8902dcce60>


In [9]:
X_train, X_dev, X_test, y_train, y_dev, y_test = train_dev_test_split(marag_df.drop(['TotalEntries'], axis=1), 
                                                                      marag_df['TotalEntries'])

X_train = pd.concat([X_train, X_dev])
X_train.sort_index(inplace=True)
y_train = pd.concat([y_train, y_dev])
y_train.sort_index(inplace=True)

cv = [(X_train, y_train)]

In [24]:
from sklearn.model_selection import GridSearchCV

rf = RandomForestRegressor(random_state = 42)
rf.fit(X_train, y_train)

/home/quark/anaconda3/envs/mpro/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=42, verbose=0, warm_start=False)

In [ ]:
(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 2, verbose=2, random_state=42, n_jobs = -1)

In [15]:
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [25]:
(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 2, verbose=2, random_state=42, n_jobs = -1)

rf_tune = GridSearchCV(rf, param_grid = random_grid, cv=cv, n_jobs=-1)

In [21]:
type(rf_tune)

sklearn.model_selection._search.GridSearchCV

In [ ]:
for i in tqdm range(4320):
    fit
    i = i+1
    

In [49]:
#groups = marag_df.groupby(marag_df['Timestamp'].dt.year).groups

#sorted_groups = [value for (key, value) in sorted(groups.items())] 


cv = [(sorted_groups[0] + sorted_groups[1], sorted_groups[2])]

In [52]:
len(cv)

1

In [37]:
cv

[(                     TotalEntries  Temperature  Precipitation  Open/Closed  \
  Timestamp                                                                    
  2017-01-16 00:00:00           NaN          NaN            NaN          NaN   
  2017-01-16 00:30:00           NaN          NaN            NaN          NaN   
  2017-01-16 01:00:00           NaN          NaN            NaN          NaN   
  2017-01-16 01:30:00           NaN          NaN            NaN          NaN   
  2017-01-16 02:00:00           NaN          NaN            NaN          NaN   
  2017-01-16 02:30:00           NaN          NaN            NaN          NaN   
  2017-01-16 03:00:00           NaN          NaN            NaN          NaN   
  2017-01-16 03:30:00           NaN          NaN            NaN          NaN   
  2017-01-16 04:00:00           NaN          NaN            NaN          NaN   
  2017-01-16 04:30:00           NaN          NaN            NaN          NaN   
  2017-01-16 05:00:00           NaN     

In [ ]:
# Determining range of estimators and max_depth
ne = range(1, 200)
md = range(1, 50)

start_RF_tune = time.time()

best_RMSE = []
for ne in tqdm(range(1, 200)):
    for md in range(1, 50):
        RF_temp_model = RandomForestRegressor(n_estimators = ne, max_depth = md, n_jobs = -1, random_state = 42)
        RF_temp_model = RF_temp_model.fit(X_train, y_train)
        y_dev_hat = RF_temp_model.predict(X_dev)
        RMSE_scoring = np.sqrt(mean_squared_error(y_dev, y_dev_hat))
        
        if pd.DataFrame(best_RMSE).empty:
            best_RMSE = [ne, md, RMSE_scoring]
            
        else:
            if RMSE_scoring > np.array(best_RMSE)[2]:
                best_RMSE = [ne, md, RMSE_scoring]

end_RF_tune = time.stime()
RF_tuning_runtime = end_RF_tune - start_RF_tune


 46%|████▌     | 92/199 [26:50:09<59:39:36, 2007.26s/it]